In [1]:
import os
import pathlib
import tempfile

from voidfindertk import io,zobov

In [7]:
# This path is used to ensure that this example works anywhere
this_file_path = pathlib.Path(os.path.abspath(''))

#Path to the dataset file
this_file_parent = this_file_path.parents[0]
path_dataset = this_file_path/'voidFinderProject'/'datasets'/'halos_fede.dat'

#Generate Directory to place the runs
temp_dir = tempfile.mkdtemp(prefix='run',dir=str(this_file_parent))

# Create data DataBox:
The DataBox object holds the data and their properties. To create this object use io module:

- io.read_table(path_or_buffer, **kwargs) : Reads a csv data input with mandatory x,y,z ,vx,vy,vz, m columns:
    - x,y,z : position of the tracers
    - vx,vy,vz : velocities of the tracers
    - m : mass of the tracers
    For kwargs arguments take reference of pandas.read_csv. Default values are:
    - usecols = [0,1,2,3,4,5,6] first 7 columns in the file
    - names = [x,y,z,vx,vy,vz,m] names asociated to those columns.

- io.xyz_read_table(path_or_buffer, **kwargs): Same as the aforementioned function, but uses only x,y,z positions (in case the datafile only contains only those tree set of columns).

From this object we can obtain a box kind object with the properties of the dataset. A useful propertie is:
- box = DataBox.box
- box.size() : returns the max lenght value of the cubic box

In [8]:
# Get DataBox object:
databox = io.read_table(path_or_buffer=path_dataset)

# Get the properties box
box = databox.box
size = box.size()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jorgefederico/updates/test_vftk/test2/datasets/halos_fede.dat'

## Running ZOBOV

To run ZOBOV first create the model, the model depends on values needed to run the finder. The input values are divided into two groups:
1. ZOBOV input params (can be found in the original docs of ZOBOV):

    - *buffer_size*: he buffer size sets the size in units such that the box size of the data cube is 1, of the buffer around each sub-box when calculating the Voronoi diagram. (Default 0.08)

    - *box_size*: Range of positions of particles in each dimension (default is 500).

    - *number_of_divisions*: the no. of partitions in each dimension; must be at least 2 (giving 8 sub-boxes)
    (default 2)

    - *density_threshold*: The density threshold is an optional parameter, which can limit the
    growth of voids into high-density regions. (default is 0).

2. Extra parameters related to work dir paths, most importants are
    - workdir: Directory specified by the user where all the finder files are going to be placed.
    - workdir_clean: Whether or not delete the working directory


In [ ]:
#Generate the model
zobov_0 = zobov.ZobovVF(
    workdir = str(str(pathlib.Path(temp_dir))),
    box_size = size,
    )

The search of voids over the dataset is performed calling the method find of the model, and providing the databox object. As a return from this proces a Void object will be obtained. The properties of this object are the following:

<div style="color:red">method<div style="color:white">: Method used to get this output, in this case ZobovVF</div></div>
<div style="color:cyan">tracers<div style="color:white">: The input box of tracers</div></div>
<div style="color:yellow">tracers_in_voids_<div style="color:white">: tuple of array of indexes of the tracers within each void</div></div>
<div style="color:green">extra_
    <div style="color:white">"zobov_path": location of original ZOBOV binaries</div>
    <div style="color:white">"zobov_voids": collection of objects VoidProperties with the properties of each void (these properties are the ones obtained in the final output of ZOBOV)</div>
    <div style="color:white">"files_dir": directory with the location of the files of this run</div>
</div>
<div style="color:violet">number_of_voids_<div style="color:white">: Integer showing the number of voids found</div></div>

In [12]:
# Perform Void Finder method on dataset
voids = zobov_0.find(databox)

Datos guardados en /home/jorgefederico/updates/test_vftk/voidFinderProject/runzeupr01n/tmpi15_gua62024-07-22T04:09:38.446825+00:00/part_vs_zone_ascii.txt
Datos guardados correctamente en /home/jorgefederico/updates/test_vftk/voidFinderProject/runzeupr01n/tmpi15_gua62024-07-22T04:09:38.446825+00:00/zones_vs_voids_ascii.txt


In [14]:
voids.voids_[0]

array([569431,   1190,   1253, ..., 419587, 449486, 691259])

In [15]:
voids.method

'ZobovVF'

In [17]:
voids.extra_['zobov_voids'][0].core_particle

569431

In [18]:
voids.tracers

<Box size=700070>

## Accesing ZOBOV raw output files
Interested in the raw files can view them inside a directory within the following directory:

directory = voids.extra_['files_dir']

- The default suffix of the files (see voztie docummentation) is output_vozinit
- The default name of the txt file is: output_txt.dat

In [21]:
# Get directory of runs of the model
model_directory = str(zobov_0.workdir)

# Directory of this specific run
directory = voids.extra_['files_dir']
directory = pathlib.Path(directory)


'/home/jorgefederico/updates/test_vftk/voidFinderProject/runzeupr01n'

Can also display the txt file as a dataframe:

In [ ]:
# Get the output txt file as a pandas dataframe
import pandas as pd
df = pd.read_csv(directory / "output_txt.dat", sep='\s+',header=1)